<span style="float:left;">Licence CC BY-NC-ND</span><span style="float:right;">François Rechenmann &amp; Thierry Parmentelat&nbsp;<img src="media/inria-25.png" style="display:inline"></span><br/>

# Recalage par recherche des RBS

Dans ce complément, nous allons implémenter l'algorithme de recalage par RBS. Mais comme d'habitude&nbsp;:

In [ ]:
# la formule magique pour utiliser print() en python2 et python3
from __future__ import print_function
# pour que la division se comporte en python2 comme en python3
from __future__ import division

### Chercher *Start* après un RBS

Pour l'essentiel, nous allons tout simplement reprendre la fonction `coding_regions_one_phase_rbs`, mais en la modifiant légèrement de façon à utiliser cette nouvelle heuristique au moment de trouver le début de la région codante. Pour cela nous commençons par écrire une fonction `next_start_codon_after_rbs`, qui va se substituer à `next_start_codon`&nbsp;:

In [ ]:
start_codon = "ATG"

def next_start_codon_after_rbs(arn, index, rbs):
    """
    recherche le premier start après une séquence RBS
    renvoie l'index du START, ou None si on n'en trouve plus
    """
    # on cherche le premier RBS
    index = arn.find(rbs, index)
    if index <= 1:
        return None
    index = arn.find(start_codon, index)
    if index <= 1:
        return None
    return index

### Variante des régions codantes, avec RBS

Ceci étant acquis, il ne nous reste plus qu'à modifier `coding_regions_one_phase` comme suit. Vous remarquerez qu'une seule ligne est modifiée par rapport à notre version simple&nbsp;:

In [ ]:
# la fonction next_stop_codon
from w3_s03_c2_next_codon import next_stop_codon

# avec à nouveau une longueur minimale de 300 par défaut
# et en utilisant le recalage RBS
def coding_regions_one_phase_rbs(adn, phase, rbs, minimal_length=300):
    # on initialise index à la phase; avec next_start_codon
    # et next_stop_codon on reste toujours sur la même phase
    index = phase
    # les résultats sont retournés sous la forme d'une liste 
    # de couples [start_gene, stop_gene]
    genes = []
    # on calcule stop1 qui désigne pour nous le stop "de gauche"
    # à ce stade, c'est le premier STOP à partir de la phase
    stop1 = next_stop_codon(adn, index)
    # s'il n'y a pas du tout de stop dans toute la séquence
    # on a terminé
    if not stop1:
        return genes
    # la boucle principale
    while True:      
        # on cherche le premier STOP à partir de stop1
        # pour trouver le stop "de droite"
        stop2 = next_stop_codon(adn, stop1+3)
        # s'il n'y a plus de stop, on a fini
        if not stop2:
            return genes
        # mais il faut qu'il soit assez loin
        if stop2 - stop1 < minimal_length:
            # c'est trop court, on saute ce fragment
            stop1 = stop2
            continue
        # à ce stade on a trouvé un ORF, reste à trouver le bon start
        ##########
        # c'est ici la seule différence avec la version simple
        # de coding_regions_one_phase
        start = next_start_codon_after_rbs(adn, stop1, rbs)
        ##########
        # s'il n'y en a pas: c'est qu'on ne trouvera plus rien
        # et donc on a fini
        if not start:
            return genes
        # ici c'est un peu subtil
        # tout d'abord rien ne nous dit qu'il y a un codon START entre start1 et start2
        # et même s'il y en a un, rien de nous dit qu'on obtient un gène assez long
        # donc on reteste la longueur du gène trouvé et on ne garde que ceux 
        # qui sont assez longs - par rapport à minimal_length
        if stop2 - start < minimal_length:
            pass
        else:
            # cette fois on a trouvé un gène, on l'ajoute dans les résultats
            genes.append( [start, stop2] )
        # on peut passer à l'ORF suivant
        stop1 = stop2

### Des données réelles

Nous allons voir le résultat des deux versions de `regions_codantes` sur une bactérie réelle, la célèbre *Escherichia Coli*, que vous pouvez consulter sur ENA sous la clé `U00096`, mais que nous avons téléchargé pour vous&nbsp;:

In [ ]:
with open("data/escherichia-coli-U00096") as input:
    e_coli = input.read()
print("Escheria Coli a {} nucléotides".format(len(e_coli)))

Si on importe l'algorithme simple&nbsp;:

In [ ]:
# la recherche de régions codantes sur une phase
# telle que nous l'avons vue dans la séquence 2
from w3_s02_c1_regions_codantes_v1 import coding_regions_one_phase

Il nous donne avec cet exemple&nbsp;:

In [ ]:
for phase in 0, 1, 2:
    algo_simple = coding_regions_one_phase(e_coli, phase)
    print("L'algorithme simple trouve {} regions sur la phase phase {}"
          .format(len(algo_simple), phase))

Si maintenant nous utilisons la séquence `AGGAGG` comme RBS, nous obtenons cette fois&nbsp;:

In [ ]:
rbs_coli = 'AGGAGG'
for phase in 0, 1, 2:
    algo_rbs = coding_regions_one_phase_rbs(e_coli, phase, rbs_coli)
    print("L'algorithme avec RBS trouve {} regions sur la phase {}"
          .format(len(algo_rbs), phase))

In [ ]:
import re
all = re.findall(rbs_coli, e_coli)
print("Trouvé le RBS en tout {} fois".format(len(all)))